## Melon music data scraping from web
* 1. Scrap the title and id of 100 songs
* 2. Request 100 times(for each detail pages) >> get detail info
* 3. Store that to pandas data frame
* 4. and store that to DB as well (song table)

In [1]:
import requests, re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
# https://www.melon.com/album/detail.htm?albumId=10462799
# >>
# songId = 32790516
# >>
# https://www.melon.com/song/detail.htm?songId=32790516
url = 'https://www.melon.com/chart/index.htm'
req_header = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36',
}
html = requests.get(url, headers = req_header).text
# print(html)
soup = BeautifulSoup(html, 'html.parser')
soup.select("tbody tr td a")
# for body in soup.find_all('tbody'):
#     for tr in body.find_all("tr"):
#         for td in tr.find_all('td'):
#             for a in td.find_all('a'):
#                 print(a.string)
#                 # print(a_title)

# print(len(soup.select('#tb_list')))
# print(len(soup.select("#tb_list tr div.wrap_song_info a[href*='playSong']")))
links = []
for songs in soup.select("#tb_list tr div.wrap_song_info a[href*='playSong']"):
    links.append({
        'title': songs.string,
        'url' : "https://www.melon.com/song/detail.htm?songId=" + songs['href'][42:-2],
    })
    ## or use RE
    # href = songs['href']
    # matched = re.search(r"(\d+)\)",href)
    # print(matched.group(1))
# print(links)
song_lyric_lists = []
song_lyric_dict = {}
for link in links:
    html = requests.get(link['url'], headers= req_header).text
    soup = BeautifulSoup(html, 'html.parser')
    # print(soup)
    song_lyric_dict['곡명'] = link['title']
    song_lyric_dict['가수'] = soup.select('a[href*=".goArtistDetail"] span')[0].text
    song_lyric_dict['앨범'] = soup.select('div.meta dd')[0].text
    song_lyric_dict['발매일'] = soup.select('div.meta dd')[1].text
    song_lyric_dict['장르'] = soup.select('div.meta dd')[2].text
    #song_lyric_dict['좋아요'] = soup.select('span#d_like_count')[0].text
    lyric = soup.select('div#d_video_summary')[0].text
    regex = re.compile(r'[\n\r\t]')
    song_lyric_dict['가사'] = regex.sub('', lyric.strip())
    song_lyric_lists.append(song_lyric_dict)
    #print(song_lyric_dict)
    song_lyric_dict = {}
print(len(song_lyric_lists))


100


In [5]:
# print(song_lyric_lists[0])
# import json
# json.dump(song_lyric_lists,)


# import re
# song_lyric_lists = []
# song_lyric_dict = {}
# for song in song_detail_lists:
#     html = requests.get(song['url'], headers=headers).text
#     soup = BeautifulSoup(html,'html.parser')
#     song_lyric_dict['곡명'] = song['title']
#     song_lyric_dict['가수'] = soup.select('a[href*=".goArtistDetail"] span')[0].text
#     song_lyric_dict['앨범'] = soup.select('div.meta dd')[0].text
#     song_lyric_dict['발매일'] = soup.select('div.meta dd')[1].text
#     song_lyric_dict['장르'] = soup.select('div.meta dd')[2].text
#     #song_lyric_dict['좋아요'] = soup.select('span#d_like_count')[0].text
#     lyric = soup.select('div#d_video_summary')[0].text
#     regex = re.compile(r'[\n\r\t]')
#     song_lyric_dict['가사'] = regex.sub('', lyric.strip())
#     song_lyric_lists.append(song_lyric_dict)
#     #print(song_lyric_dict)
#     song_lyric_dict = {}
# print(len(song_lyric_lists))

{'곡명': '다시 여기 바닷가', '가수': '싹쓰리 (유두래곤, 린다G, 비룡)', '앨범': '다시 여기 바닷가', '발매일': '2020.07.18', '장르': '댄스', '가사': '예아 호우 예예예싹쓰리 인더 하우스커커커커커몬 싹 쓰리 투 렛츠고나 다시 또 설레어이렇게 너를 만나서함께 하고 있는 지금 이 공기가다시는 널 볼 순 없을 거라고추억일 뿐이라서랍 속에 꼭 넣어뒀는데흐르는 시간 속에서너와 내 기억은점점 희미해져만 가끝난 줄 알았어지난여름 바닷가너와 나 단둘이파도에 취해서 노래하며같은 꿈을 꾸었지다시 여기 바닷가이제는 말하고 싶어네가 있었기에 내가 더욱 빛나별이 되었다고다들 덥다고 막 짜증내괜찮아 우리 둘은 따뜻해내게 퐁당 빠져버린 널이젠 구하러 가지 않을 거야모래 위 펴펴펴편지를 써밀물이 밀려와도 못 지워추억이 될 뻔한 첫 느낌너랑 다시 한번 받아 보고 싶어흐르는 시간 속에서너와 내 기억은점점 희미해져만 가끝난 줄 알았어지난여름 바닷가너와 나 단둘이파도에 취해서 노래하며같은 꿈을 꾸었지다시 여기 바닷가이제는 말하고 싶어네가 있었기에 내가 더욱 빛나별이 되었다고시간의 강을 건너또 맞닿은 너와 나소중한 사랑을 영원히간직해줘지난여름 바닷가너와 나 단둘이파도에 취해서 노래하며같은 꿈을 꾸었지다시 여기 바닷가이제는 말하고 싶어네가 있었기에 내가 더욱 빛나별이 되었다고'}
